# Lab 3B: Motion Feature Extraction
**Course:** AIML ZG531 - Video Analysis  
**Module:** 3 - Feature Extraction  
**Topic:** Temporal Motion Features Using Frame Differencing, Background Subtraction, and Optical Flow  
**Author:** Seetha Parameswaran

---

## Learning Objectives
- Extract motion information using frame differencing techniques
- Implement background subtraction for foreground detection
- Compute sparse optical flow using Lucas-Kanade method
- Calculate dense optical flow using Farneback algorithm
- Visualize and interpret motion patterns in videos
- Understand the brightness constancy assumption and aperture problem

This script extracts motion features:
1. Frame Differencing
2. Background Subtraction (GMM)
3. Optical Flow (Lucas-Kanade, Farneback) 

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os


class MotionFeatureExtractor:
    """Extract motion features from video"""
    
    def __init__(self, video_path):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(video_path)
        
        if not self.cap.isOpened():
            raise ValueError(f"Cannot open video: {video_path}")
        
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
        self.total_frames = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        print(f"Video: {self.width}x{self.height}, {self.fps} fps, {self.total_frames} frames")
    
    def frame_differencing(self, frame1, frame2, threshold=30):
        """
        Compute frame difference
        
        Args:
            frame1, frame2: Consecutive frames
            threshold: Difference threshold
        Returns:
            Binary difference mask
        """
        gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        
        diff = cv2.absdiff(gray1, gray2)
        _, thresh = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
        
        return diff, thresh
    
    def background_subtraction(self, frames, learning_rate=0.001):
        """
        Background subtraction using MOG2
        
        Args:
            frames: List of frames
            learning_rate: Learning rate for background model
        Returns:
            Foreground masks
        """
        bg_subtractor = cv2.createBackgroundSubtractorMOG2(
            history=500, varThreshold=16, detectShadows=True
        )
        
        masks = []
        for frame in frames:
            mask = bg_subtractor.apply(frame, learningRate=learning_rate)
            masks.append(mask)
        
        return masks
    
    def optical_flow_lucas_kanade(self, frame1, frame2, max_corners=100):
        """
        Sparse optical flow using Lucas-Kanade
        
        Args:
            frame1, frame2: Consecutive frames
            max_corners: Max feature points
        Returns:
            Feature points and flow vectors
        """
        gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        
        # Detect corners in first frame
        p0 = cv2.goodFeaturesToTrack(
            gray1, maxCorners=max_corners, 
            qualityLevel=0.01, minDistance=10
        )
        
        if p0 is None:
            return None, None
        
        # Calculate optical flow
        p1, status, _ = cv2.calcOpticalFlowPyrLK(gray1, gray2, p0, None)
        
        # Select good points
        good_old = p0[status == 1]
        good_new = p1[status == 1]
        
        return good_old, good_new
    
    def optical_flow_farneback(self, frame1, frame2):
        """
        Dense optical flow using Farneback method
        
        Args:
            frame1, frame2: Consecutive frames
        Returns:
            Flow field (u, v components)
        """
        gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        
        flow = cv2.calcOpticalFlowFarneback(
            gray1, gray2, None,
            pyr_scale=0.5, levels=3, winsize=15,
            iterations=3, poly_n=5, poly_sigma=1.2, flags=0
        )
        
        return flow
    
    def visualize_motion_features(self, frame1, frame2, save_path=None):
        """Visualize all motion features"""
        
        fig = plt.figure(figsize=(16, 10))
        
        # Original frames
        ax1 = plt.subplot(3, 3, 1)
        ax1.imshow(cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB))
        ax1.set_title('Frame t')
        ax1.axis('off')
        
        ax2 = plt.subplot(3, 3, 2)
        ax2.imshow(cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB))
        ax2.set_title('Frame t+1')
        ax2.axis('off')
        
        # Frame differencing
        diff, thresh = self.frame_differencing(frame1, frame2)
        ax3 = plt.subplot(3, 3, 3)
        ax3.imshow(diff, cmap='hot')
        ax3.set_title('Frame Difference')
        ax3.axis('off')
        
        ax4 = plt.subplot(3, 3, 4)
        ax4.imshow(thresh, cmap='gray')
        ax4.set_title('Thresholded Difference')
        ax4.axis('off')
        
        # Background subtraction
        masks = self.background_subtraction([frame1, frame2])
        ax5 = plt.subplot(3, 3, 5)
        ax5.imshow(masks[1], cmap='gray')
        ax5.set_title('Background Subtraction')
        ax5.axis('off')
        
        # Lucas-Kanade optical flow
        p0, p1 = self.optical_flow_lucas_kanade(frame1, frame2)
        ax6 = plt.subplot(3, 3, 6)
        frame_lk = cv2.cvtColor(frame2.copy(), cv2.COLOR_BGR2RGB)
        if p0 is not None and p1 is not None:
            for (x0, y0), (x1, y1) in zip(p0, p1):
                cv2.arrowedLine(frame_lk, (int(x0), int(y0)), 
                              (int(x1), int(y1)), (0, 255, 0), 2, tipLength=0.3)
        ax6.imshow(frame_lk)
        ax6.set_title(f'Lucas-Kanade Flow ({len(p0) if p0 is not None else 0} points)')
        ax6.axis('off')
        
        # Farneback dense optical flow
        flow = self.optical_flow_farneback(frame1, frame2)
        
        # Flow magnitude
        magnitude = np.sqrt(flow[..., 0]**2 + flow[..., 1]**2)
        ax7 = plt.subplot(3, 3, 7)
        ax7.imshow(magnitude, cmap='jet')
        ax7.set_title('Flow Magnitude')
        ax7.axis('off')
        
        # Flow direction (HSV visualization)
        hsv = np.zeros((flow.shape[0], flow.shape[1], 3), dtype=np.uint8)
        hsv[..., 1] = 255
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
        
        ax8 = plt.subplot(3, 3, 8)
        ax8.imshow(rgb)
        ax8.set_title('Flow Direction (HSV)')
        ax8.axis('off')
        
        # Flow statistics
        ax9 = plt.subplot(3, 3, 9)
        stats_text = f"Flow Statistics:\n"
        stats_text += f"Mean magnitude: {np.mean(magnitude):.2f}\n"
        stats_text += f"Max magnitude: {np.max(magnitude):.2f}\n"
        stats_text += f"Motion pixels (>1): {np.sum(magnitude > 1)}\n"
        ax9.text(0.1, 0.5, stats_text, fontsize=10, verticalalignment='center')
        ax9.axis('off')
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            print(f"Saved: {save_path}")
        
        plt.show()
    
    def process_video_sample(self, sample_pairs=5, output_dir='motion_features'):
        """Process sample frame pairs"""
        
        os.makedirs(output_dir, exist_ok=True)
        
        # Sample frame pairs
        frame_indices = np.linspace(0, self.total_frames - 10, sample_pairs, dtype=int)
        
        print(f"\nProcessing {sample_pairs} frame pairs...")
        
        for i, frame_idx in enumerate(frame_indices):
            print(f"\nPair {i+1}/{sample_pairs}: frames {frame_idx} and {frame_idx+1}")
            
            # Read consecutive frames
            self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            ret1, frame1 = self.cap.read()
            ret2, frame2 = self.cap.read()
            
            if not (ret1 and ret2):
                print(f"Cannot read frames {frame_idx}")
                continue
            
            # Visualize
            save_path = os.path.join(output_dir, f'motion_frame_{frame_idx:06d}.png')
            self.visualize_motion_features(frame1, frame2, save_path)
    
    def __del__(self):
        if hasattr(self, 'cap'):
            self.cap.release()




In [ ]:
def main():
    video_path = "vid.avi"  # Replace with your video
    
    print("=" * 60)
    print("Motion Feature Extraction")
    print("Module 3: Video Analysis")
    print("=" * 60)
    
    try:
        extractor = MotionFeatureExtractor(video_path)
        extractor.process_video_sample(sample_pairs=5)
        
        print("\n" + "=" * 60)
        print("Completed!")
        print("=" * 60)
        
    except Exception as e:
        print(f"\nError: {str(e)}")
        print("\nInstall: pip install opencv-python matplotlib numpy")


if __name__ == "__main__":
    main()

Motion Feature Extraction
Module 3: Video Analysis

Error: Cannot open video: /home/seetha/PythonScriptsCourses/PythonScriptsCourses/Video Analytics/data/Life by the river_1080p.mp4

Install: pip install opencv-python matplotlib numpy


## Summary and Key Takeaways

### Motion Extraction Methods

1. **Frame Differencing**
   - Simplest motion detection: |I(t) - I(t-1)|
   - Fast, low computational cost
   - Limitations: Sensitive to noise, no velocity information
   - Use case: Basic motion detection, trigger systems

2. **Background Subtraction (MOG2)**
   - Gaussian Mixture Model for background modeling
   - Adapts to gradual changes (lighting, weather)
   - Handles shadows (detectShadows parameter)
   - Use case: Surveillance, people counting, abandoned object detection

3. **Lucas-Kanade (Sparse Flow)**
   - Tracks specific feature points
   - Assumes brightness constancy and small motion
   - Computationally efficient
   - Use case: Object tracking, structure from motion

4. **Farneback (Dense Flow)**
   - Computes flow for every pixel
   - Polynomial expansion approach
   - More computationally intensive
   - Use case: Video stabilization, motion segmentation, action recognition

### Motion Representation
- **Magnitude:** Speed of motion
- **Direction:** Angle of motion (HSV visualization)
- **Flow Vectors:** Complete motion field description

---

## Exercise Questions

1. **Conceptual**: Explain why the brightness constancy assumption fails in real-world scenarios. Give three specific examples from different application domains.

2. **Analysis**: Compare the number of motion pixels detected by frame differencing vs. background subtraction. Which method has fewer false positives? Why?

3. **Application**: For autonomous vehicle pedestrian detection, would you use sparse or dense optical flow? Consider real-time constraints and safety requirements.

4. **Implementation**: Modify the optical flow code to detect only large motions (magnitude > threshold). How does this affect the detection of different types of activities?

5. **Critical Thinking**: The aperture problem causes ambiguity in optical flow. Design a strategy to detect and handle regions affected by this problem.

6. **Algorithmic**: Background subtraction adapts with a learning rate. Experiment with different learning rates (0.0001, 0.01, 0.1). How does this affect detection of stationary vs. moving objects?